In [9]:
# Code for permanent library installation & import
from google.colab import drive
import sys
import os
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
paths = [
    '/content/drive/MyDrive/Colab_env/lib/python3.10/site-packages',
    '/content/drive/MyDrive/Colab_env/benchmarks/speechbrain'
]

# Get the current PYTHONPATH
current_pythonpath = os.environ.get('PYTHONPATH', '')

# Create a new PYTHONPATH including your custom paths
new_pythonpath = ':'.join(paths) if current_pythonpath == '' else ':'.join([current_pythonpath] + paths)

# Set the new PYTHONPATH in the environment
os.environ['PYTHONPATH'] = new_pythonpath

sys.path.append('/content/drive/MyDrive/Colab_env/lib/python3.10/site-packages')
sys.path.append('/content/drive/MyDrive/Colab_env/benchmarks/speechbrain')

In [ ]:
# %%capture
# %cd /content/drive/MyDrive/Colab_env
# !git clone https://github.com/speechbrain/benchmarks.git
# %cd benchmarks
# !git submodule update --init --recursive
# %cd speechbrain
# !pip install -r requirements.txt
# !pip install -e .
# %cd /content/benchmarks/benchmarks/MOABB
# !pip install -r ../../requirements.txt    # Install base dependencies
# !pip install -r extra-requirements.txt    # Install additional dependencies

# paths = [
#     '/content/drive/MyDrive/Colab_env/lib/python3.10/site-packages',
#     '/content/drive/MyDrive/Colab_env/benchmarks/speechbrain'
# ]

# # Get the current PYTHONPATH
# current_pythonpath = os.environ.get('PYTHONPATH', '')

# # Create a new PYTHONPATH including your custom paths
# new_pythonpath = ':'.join(paths) if current_pythonpath == '' else ':'.join([current_pythonpath] + paths)

# # Set the new PYTHONPATH in the environment
# os.environ['PYTHONPATH'] = new_pythonpath

In [ ]:
# train EEGNet for Motor Imagery using the BNCI2014001 dataset. Specifically, we will train the model using data from subject 0. The data recorded in session 1 of subject 0 will be used for testing, while all the other sessions will be used for training.
# !python train.py hparams/MotorImagery/BNCI2014001/EEGNet.yaml --data_folder=eeg_data --cached_data_folder=eeg_pickled_data --output_folder=results/MotorImagery/BNCI2014001/ --target_subject_idx=0 --target_session_idx=1 --data_iterator_name=leave-one-session-out --device='cpu'

###**Load the Inner speech database**

In [3]:
%cd /content/drive/MyDrive/Colab_env/
#@title Imports
import mne
import warnings
import numpy as np
from Inner_Speech_Dataset.Python_Processing.Data_extractions import  extract_data_from_subject
from Inner_Speech_Dataset.Python_Processing.Data_processing import  select_time_window, transform_for_classificator

np.random.seed(23)

mne.set_log_level(verbose='warning') #to avoid info at terminal
warnings.filterwarnings(action = "ignore", category = DeprecationWarning )
warnings.filterwarnings(action = "ignore", category = FutureWarning )

/content/drive/MyDrive/Colab_env


In [6]:
### Hyperparameters

# The root dir have to point to the folder that cointains the database
root_dir = "/content/drive/MyDrive/Colab_env/eeg_data"

# Data Type
datatype = "EEG"

# Sampling rate
fs = 256

# Select the useful par of each trial. Time in seconds
t_start = 1.5
t_end = 3.5

# Subject number
N_S = 1   #[1 to 10]

In [7]:
#@title Data extraction and processing

# Load all trials for a sigle subject
X, Y = extract_data_from_subject(root_dir, N_S, datatype)

# Cut usefull time. i.e action interval
X = select_time_window(X = X, t_start = t_start, t_end = t_end, fs = fs)

In [8]:
print("Data shape: [trials x channels x samples]")
print(X.shape) # Trials, channels, samples

print("Labels shape")
print(Y.shape) # Time stamp, class , condition, session

Data shape: [trials x channels x samples]
(500, 128, 512)
Labels shape
(500, 4)


In [10]:
# Conditions to compared
Conditions = [["Inner"],["Inner"]]
# The class for the above condition
Classes    = [  ["Up"] ,["Down"] ]

In [11]:
# Transform data and keep only the trials of interes
X , Y =  transform_for_classificator(X, Y, Classes, Conditions)

In [12]:
print("Final data shape")
print(X.shape)

print("Final labels shape")
print(Y.shape)

Final data shape
(100, 128, 512)
Final labels shape
(100,)
